### Header

In [ ]:
# import libraries

# maths
import numpy as np
import pandas as pd
import scipy.stats as stats
from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

# modelling
from sklearn import linear_model
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline,Pipeline

# nlp
#from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB
#from nltk.stem import PorterStemmer,WordNetLemmatizer
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.corpus import stopwords
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import spacy
#from spacy.tokens import Doc

# web
#import requests
#import json

# others
import os
import re
import time
import datetime as datetime

In [ ]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Functions

### Import Data

In [ ]:
# import clean data

df_train = pd.read_csv(clean_path + 'train_clean.csv')
df_test = pd.read_csv(clean_path + 'test_clean.csv')
street_map = gpd.read_file('../data/2_input/Chicago Boundaries/geo_export_d41dc94d-31ce-478c-902c-864695385885.shp')
#df_weather = pd.read_csv(clean_path + 'weather_clean.csv')
df_spray = pd.read_csv(clean_path + 'spray_clean.csv')

### EDA

In [ ]:
df_train.head()

In [ ]:
train_year = list(df_train.year.unique())
train_year

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8),sharey=True)
ax = ax.ravel()
plt.suptitle('Number of mosquitoes by year', fontsize=20, y=1.05)
for i, y in enumerate(train_year):
    df_train[df_train['year']==y].groupby(['year', 'month']).nummosquitos.sum().plot(kind='bar', ax=ax[i])
    ax[i].set_title(y)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8),sharey=True)
ax = ax.ravel()
plt.suptitle('Number of viruses by year', fontsize=20, y=1.05)
for i, y in enumerate(train_year):
    df_train[df_train['year']==y].groupby(['year', 'month']).wnvpresent.sum().plot(kind='bar', ax=ax[i])
    ax[i].set_title(y)
plt.tight_layout()

In [ ]:
sns.scatterplot(x=df_train.groupby(['year', 'month']).nummosquitos.sum(),
                y= df_train.groupby(['year', 'month']).wnvpresent.sum(),
               data=df_train)

In [ ]:
df_spray.head()

In [ ]:
spray_year = list(df_spray.year_spray.unique())
spray_year

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,5),sharey=True)
ax = ax.ravel()
plt.suptitle('Spray by year', fontsize=20, y=1.05)
for i, y in enumerate(spray_year):
    df_spray[df_spray['year_spray']==y].groupby(['year_spray', 'month_spray']).latitude.count().plot(kind='bar', ax=ax[i])
    ax[i].set_title(y)
    ax[i].set_xlabel('Year and month sprayed')
plt.tight_layout()

In [ ]:
crs = {'init': 'epsg:4326'}
df_train.head()

In [ ]:
geometry_train = [Point(xy) for xy in zip(df_train['longitude'], df_train['latitude'])]

geo_df = gpd.GeoDataFrame(df_train,
                      crs=crs,
                      geometry=geometry_train)
geo_df.head()

In [ ]:
geometry_spray = [Point(xy) for xy in zip(df_spray['longitude'], df_spray['latitude'])]
geo_spray_df = gpd.GeoDataFrame(df_spray,
                      crs=crs,
                      geometry=geometry_spray)
geo_spray_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax, alpha=0.4, color='grey')
geo_df[geo_df['wnvpresent']==0].plot(ax=ax, markersize=20, color='blue', marker='o', label='Neg')
geo_df[geo_df['wnvpresent']==1].plot(ax=ax, markersize=20, color='red', marker='o', label='Pos')
plt.legend(prop={'size': 15})

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax=ax, alpha=0.4, color='grey')
geo_df[geo_df['wnvpresent']==0].plot(ax=ax, markersize=20, color='blue', marker='o', label='Neg')
geo_df[geo_df['wnvpresent']==1].plot(ax=ax, markersize=20, color='red', marker='o', label='Pos')
geo_spray_df.plot(ax=ax, markersize=20, color='green', marker='o', label = 'Spray')
plt.legend(prop={'size': 15})